In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from datetime import date

In [3]:
data = pd.read_csv("raw.csv", sep=',', parse_dates={'date' : ['time']}, infer_datetime_format=True, index_col='date')

In [4]:
power_consumption = data.i_m1sum - data.i_meterfeed - data.i_power - +data.i_metercons - data.i_boostpower #+ data.i_m2sum
data["power_consumption"] = power_consumption

print("Min: ",power_consumption.min())
print("Max: ",power_consumption.max())
print("Mean: ",power_consumption.mean())
print("Std: ",power_consumption.std())

Min:  -622.0
Max:  1272.0
Mean:  75.55829363360762
Std:  122.42052544712475


In [7]:
factor_temperature = 10
liter_in_tank = 500
kw_pro_liter = 1.16
const_03 = 0.3



data["on_off"] = np.where(data['i_power'] > 0, 1, 0)

temp_diff_off = data[data.on_off == 0][["i_temp1", "i_temp2", "i_temp3"]].diff()
print(temp_diff_off.median())

m_diff = (temp_diff_off.i_temp1 + temp_diff_off.i_temp2 + temp_diff_off.i_temp3) / 3 / factor_temperature

idx_temp = m_diff[m_diff < 0].index
data["thermal_consumption"] = 0
data.thermal_consumption[idx_temp] = m_diff[m_diff < 0]

leistung_pro_grad = liter_in_tank * kw_pro_liter /1000 * const_03

data["thermal_consumption_kwh"] = data.thermal_consumption * leistung_pro_grad 
data["mean_temperature"] = (data.i_temp1 + data.i_temp2 + data.i_temp3) / 3 /factor_temperature
data["kwh_eq_state"] = data.mean_temperature*leistung_pro_grad


data

i_temp1   -1.0
i_temp2   -1.0
i_temp3   -1.0
dtype: float64


C:\Users\lukas\AppData\Local\Temp\ipykernel_28108\2238817971.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.thermal_consumption[idx_temp] = m_diff[m_diff < 0]
C:\Users\lukas\AppData\Local\Temp\ipykernel_28108\2238817971.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.thermal_consumption[idx_temp] = m_diff[m_diff < 0]


,Unnamed: 0,id,serial,last_modified_utc,i_power,i_boostpower,i_meterfeed,i_metercons,i_temp1,i_m0l1,...,i_temp4,exported,power_consumption,on_off,thermal_consumption,thermal_consumption_kwh,mean_temperature,kwh_eq_state,demand_price,feedin_price
date,,,,,,,,,,,,,,,,,,,,,
2020-10-16 07:00:00,0,5836000,2.001000e+15,10/16/2020 7:00,0,0,0,-905,362.0,NaN,...,NaN,0,NaN,0,0.000000,0.0000,NaN,NaN,0.45,0.15
2020-10-16 07:15:00,1,5836583,2.001000e+15,10/16/2020 7:15,0,0,0,-772,359.0,NaN,...,NaN,0,NaN,0,0.000000,0.0000,NaN,NaN,0.45,0.15
2020-10-16 07:30:00,2,5837152,2.001000e+15,10/16/2020 7:30,0,0,5,-100,296.0,NaN,...,NaN,0,NaN,0,0.000000,0.0000,NaN,NaN,0.45,0.15
2020-10-16 07:45:00,3,5837764,2.001000e+15,10/16/2020 7:45,0,0,0,-180,287.0,NaN,...,NaN,0,NaN,0,0.000000,0.0000,NaN,NaN,0.45,0.15
2020-10-16 08:00:00,4,5838293,2.001000e+15,10/16/2020 8:00,1,0,11,-9,288.0,NaN,...,NaN,0,NaN,1,0.000000,0.0000,NaN,NaN,0.45,0.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-19 04:30:00,50836,152977606,2.001000e+15,8/19/2022 4:30,0,0,0,0,541.0,NaN,...,NaN,0,15.0,0,0.000000,0.0000,47.533333,8.2708,0.30,0.10
2022-08-19 04:45:00,50837,152983718,2.001000e+15,8/19/2022 4:45,0,0,0,0,540.0,NaN,...,NaN,0,15.0,0,-0.066667,-0.0116,47.466667,8.2592,0.30,0.10
2022-08-19 05:00:00,50838,152984407,2.001000e+15,8/19/2022 5:00,13,0,2,-2,539.0,NaN,...,NaN,0,-8.0,1,0.000000,0.0000,47.133333,8.2012,0.30,0.10


In [6]:
data["demand_price"] = 0.40


data.demand_price[(data.index.hour > 0) & (data.index.hour <= 6)  ] = 0.30
data.demand_price[(data.index.hour > 6) & (data.index.hour <= 12)  ] = 0.45
data.demand_price[(data.index.hour > 12) & (data.index.hour <= 18)  ] = 0.47
data.demand_price[(data.index.hour > 18) & (data.index.hour <= 22)  ] = 0.50
data.demand_price[data.index.hour > 22] = 0.35

data["feedin_price"] = data.demand_price / 3

C:\Users\lukas\AppData\Local\Temp\ipykernel_28108\3134414747.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.demand_price[(data.index.hour > 0) & (data.index.hour <= 6)  ] = 0.30
C:\Users\lukas\AppData\Local\Temp\ipykernel_28108\3134414747.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.demand_price[(data.index.hour > 6) & (data.index.hour <= 12)  ] = 0.45
C:\Users\lukas\AppData\Local\Temp\ipykernel_28108\3134414747.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

In [8]:
class dataset():

  def __init__ (self, data, start_date="2022-01-01", target="i_power", scale_target=False, scale_variables=True, time_features=True):
    self.df = data.copy()
    self.start_date = start_date
    self.target = target
    self.scale_target = scale_target
    self.scale_variables = scale_variables
    self.time_features_ = time_features
    self.scaler = StandardScaler()
    # self.pipeline()

  def time_features(self, data, return_te_only=False):
    data["date"] = pd.to_datetime(data.index)

    data["month"] = data["date"].dt.month
    data["day"] = data["date"].dt.day
    data["hour"] = data["date"].dt.hour
    data["weekday"] = data["date"].dt.dayofweek
    data["minute"] = data["date"].dt.minute


    del data["date"]

    if return_te_only:
      return data[["month",	"day",	"hour",	"weekday", "minute", self.target]]

    else:
      return data

  def scale_col(self, col):
    self.df[col] = self.scaler.fit_transform(self.df[col].values.reshape(-1, 1))

  def preprocess(self):
    self.df = self.df.fillna(0)
    self.df.sort_index()

  def pipeline(self):
    self.preprocess()
    self.df["power_consumption"] = (self.df.i_m1sum - self.df.i_meterfeed - self.df.i_power + self.df.i_m2sum - +self.df.i_metercons - self.df.i_boostpower).values.clip(0)
    self.df = self.df[(self.df.index > self.start_date) & (self.df.index <  "2022-08-19")]

    if self.time_features_:
      self.df = self.time_features(self.df)

    continuous_cols = ['i_power',
       'i_boostpower', 'i_meterfeed', 'i_metercons', 'i_temp1', 'i_m0l1',
       'i_m0l2', 'i_m0l3', 'i_m1sum', 'i_m1l1', 'i_m1l2', 'i_m1l3', 'i_m2sum',
       'i_m2l1', 'i_m2l2', 'i_m2l3', 'i_m2soc', 'i_m3sum', 'i_m3l1', 'i_m3l2',
       'i_m3l3', 'i_m3soc', 'i_m4sum', 'i_m4l1', 'i_m4l2', 'i_m4l3',
       'i_temp2', 'i_power1', 'i_power2', 'i_power3', 'i_temp3', 'i_temp4', "power_consumption", 	"thermal_consumption"	,"thermal_consumption_kwh"	,"demand_price"	,"energy_price"]

    continuous_cols.remove(self.target)

    if self.scale_variables:
      [self.scale_col(col) for col in continuous_cols]

    if self.scale_target:
      self.scale_col(self.target)


    # cols = list(self.df.columns)
    # cols.remove(self.target)
    # cols.append(self.target)

    # self.df = self.df.reindex(columns=cols)

    return self.df
    

In [9]:
dset = dataset(data, start_date="2022-01-01", target="thermal_consumption_kwh", scale_target=False, scale_variables=False, time_features=False)
processed_df = dset.pipeline()
processed_df

,Unnamed: 0,id,serial,last_modified_utc,i_power,i_boostpower,i_meterfeed,i_metercons,i_temp1,i_m0l1,...,i_temp4,exported,power_consumption,on_off,thermal_consumption,thermal_consumption_kwh,mean_temperature,kwh_eq_state,demand_price,feedin_price
date,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:15:00,29004,72896886,2.001000e+15,1/1/2022 0:15,4,0,0,-2,557.0,0.0,...,0.0,0,0.0,1,0.000000,0.0000,47.066667,8.1896,0.40,0.133333
2022-01-01 00:30:00,29005,72899258,2.001000e+15,1/1/2022 0:30,0,0,0,0,556.0,0.0,...,0.0,0,0.0,0,-0.333333,-0.0580,46.800000,8.1432,0.40,0.133333
2022-01-01 00:45:00,29006,72902996,2.001000e+15,1/1/2022 0:46,0,0,0,0,555.0,0.0,...,0.0,0,0.0,0,-0.433333,-0.0754,46.366667,8.0678,0.40,0.133333
2022-01-01 01:00:00,29007,72904623,2.001000e+15,1/1/2022 1:00,0,0,0,0,554.0,0.0,...,0.0,0,0.0,0,-0.100000,-0.0174,46.266667,8.0504,0.30,0.100000
2022-01-01 01:15:00,29008,72908836,2.001000e+15,1/1/2022 1:16,0,0,0,0,553.0,0.0,...,0.0,0,0.0,0,-0.033333,-0.0058,46.233333,8.0446,0.30,0.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-18 22:45:00,50813,152870777,2.001000e+15,8/18/2022 22:45,0,0,0,0,560.0,0.0,...,0.0,0,0.0,0,-0.033333,-0.0058,48.800000,8.4912,0.50,0.166667
2022-08-18 23:00:00,50814,152876427,2.001000e+15,8/18/2022 23:01,0,0,0,0,559.0,0.0,...,0.0,0,0.0,0,-0.066667,-0.0116,48.733333,8.4796,0.35,0.116667
2022-08-18 23:15:00,50815,152879638,2.001000e+15,8/18/2022 23:15,0,0,0,0,558.0,0.0,...,0.0,0,0.0,0,-0.066667,-0.0116,48.666667,8.4680,0.35,0.116667


In [10]:
processed_df.kwh_eq_state.max()

10.764799999999997

In [13]:
#export_cols = ["month", "day", "hour",	"weekday", "minute", "i_m1sum", "power_consumption"]
#export_cols = ["i_m1sum","thermal_consumption_kwh"]
#export_cols = ["i_m1sum", "i_power", "i_temp1", "i_temp2", "i_temp3"]
export_cols = processed_df.columns

export_index = True

file_name = "dset" + date.today().strftime("_%d-%m") +  ".csv"

In [14]:
processed_df[export_cols].to_csv(file_name, index=export_index)

In [ ]:
data = processed_df
((data.i_temp1 +data.i_temp2 + data.i_temp3)/3).mean()
((data.i_temp1 +data.i_temp2 + data.i_temp3)/3).std()

95.3344425268267

In [ ]:
processed_df.to_csv("df.csv")

In [ ]:
from statsmodels.tsa.stattools import adfuller
from numpy import log
X = data[["i_m1sum"]].values
result = adfuller(X)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

ADF Statistic: -28.643630
p-value: 0.000000
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567


In [ ]:
def p(data):
  print(((data.i_temp1 +data.i_temp2 + data.i_temp3)/3).mean(), ((data.i_temp1 +data.i_temp2 + data.i_temp3)/3).std())


In [ ]:
p(data[:5000])
p(data[5000:10000])
p(data[10000:15000])
p(data[15000:20000])
p(data[20000:])

376.1618 105.61625170270526
501.1975333333334 68.79608794459953
508.3096 62.171597377381275
549.1797333333334 37.64161241547773
545.0367309458219 35.96063254544149


In [ ]:
data.index = pd.DatetimeIndex(data.index)
res = data["i_m1sum"].groupby(pd.Grouper(freq='M'))
res.sum() /1000

date
2022-01-31     282.930
2022-02-28     672.712
2022-03-31    1370.504
2022-04-30    1095.415
2022-05-31    1440.642
2022-06-30    1426.572
2022-07-31    1526.441
2022-08-31     837.161
Freq: M, Name: i_m1sum, dtype: float64